In [13]:
import time

from utils.loader import FactoryLoader
from utils.ml import MLPipeline
from utils.preprocessing import PreprocessingFactory
from utils.feature_extraction import *
from utils.utils import *

VAL_PATH = r"C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\val"
TRAIN_PATH = r"C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\train"

In [14]:
percent = 100
random = False

# Initialize the FactoryLoader
factory = PreprocessingFactory()
factory.gaussian_smoothing(3)
factory.clahe(clip_limit=3)
factory.pad2square(fill=np.nan)
factory.resize((150,150))
factory.hair_removal()
factory.normalize2float()

factory_loader = FactoryLoader(path=VAL_PATH, batch_size=32, factory=factory, percentage=percent, shuffle=random)

# Create the feature extraction pipeline strategy and add desired features
strategy = FeatureExtractionStrategy()

strategy.add_extractor(MeanExtractor())  # Add mean feature
strategy.add_extractor(StdExtractor())   # Add standard deviation feature
strategy.add_extractor(VarExtractor())   # Add variance feature

strategy.add_extractor(MeanExtractor("lab"))  # Add mean feature
strategy.add_extractor(StdExtractor("lab"))   # Add standard deviation feature
strategy.add_extractor(VarExtractor("lab"))   # Add variance feature

strategy.add_extractor(MeanExtractor("hsv"))  # Add mean feature
strategy.add_extractor(StdExtractor("hsv"))   # Add standard deviation feature
strategy.add_extractor(VarExtractor("hsv"))   # Add variance feature

strategy.add_extractor(LBPExtractor(radius=1, n_points=8))
strategy.add_extractor(LBPExtractor(radius=1, n_points=16))

strategy.add_extractor(GLCMExtractor(properties=['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation',]))

pipeline = MLPipeline(dataset_path=TRAIN_PATH, preprocessing_factory=factory, feature_strategy=strategy, classifiers=[], percentage=percent, verbose=True, shuffle=random)

INFO:utils.ml:MLPipeline initialized with dataset path: C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\train
INFO:utils.ml:Preprocessing steps


In [15]:
pipeline.loader.get_transformation_steps()

{'smoothing': {'kernel_size': 3},
 'clahe': {'clip_limit': 3, 'tile_grid_size': (8, 8)},
 'pad2square': {'fill': nan},
 'resize': {'size': (150, 150)},
 'hair_removal': {},
 '01_norm': {}}

In [16]:
pipeline.get_feature_names()

['mean_rgb_channel_0',
 'mean_rgb_channel_1',
 'mean_rgb_channel_2',
 'std_rgb_channel_0',
 'std_rgb_channel_1',
 'std_rgb_channel_2',
 'var_rgb_channel_0',
 'var_rgb_channel_1',
 'var_rgb_channel_2',
 'mean_lab_channel_0',
 'mean_lab_channel_1',
 'mean_lab_channel_2',
 'std_lab_channel_0',
 'std_lab_channel_1',
 'std_lab_channel_2',
 'var_lab_channel_0',
 'var_lab_channel_1',
 'var_lab_channel_2',
 'mean_hsv_channel_0',
 'mean_hsv_channel_1',
 'mean_hsv_channel_2',
 'std_hsv_channel_0',
 'std_hsv_channel_1',
 'std_hsv_channel_2',
 'var_hsv_channel_0',
 'var_hsv_channel_1',
 'var_hsv_channel_2',
 'lbp_rad1_bins8_0',
 'lbp_rad1_bins8_1',
 'lbp_rad1_bins8_2',
 'lbp_rad1_bins8_3',
 'lbp_rad1_bins8_4',
 'lbp_rad1_bins8_5',
 'lbp_rad1_bins8_6',
 'lbp_rad1_bins8_7',
 'lbp_rad1_bins8_8',
 'lbp_rad1_bins8_9',
 'lbp_rad1_bins16_0',
 'lbp_rad1_bins16_1',
 'lbp_rad1_bins16_2',
 'lbp_rad1_bins16_3',
 'lbp_rad1_bins16_4',
 'lbp_rad1_bins16_5',
 'lbp_rad1_bins16_6',
 'lbp_rad1_bins16_7',
 'lbp_rad1_

In [17]:
pipeline.run_feature_extraction()

INFO:utils.ml:Running feature extraction...


Processed 5/634 batches.
Processed 10/634 batches.
Processed 15/634 batches.
Processed 20/634 batches.
Processed 25/634 batches.
Processed 30/634 batches.
Processed 35/634 batches.
Processed 40/634 batches.
Processed 45/634 batches.
Processed 50/634 batches.
Processed 55/634 batches.
Processed 60/634 batches.
Processed 65/634 batches.
Processed 70/634 batches.
Processed 75/634 batches.
Processed 80/634 batches.
Processed 85/634 batches.
Processed 90/634 batches.
Processed 95/634 batches.
Processed 100/634 batches.
Processed 105/634 batches.
Processed 110/634 batches.
Processed 115/634 batches.
Processed 120/634 batches.
Processed 125/634 batches.
Processed 130/634 batches.
Processed 135/634 batches.
Processed 140/634 batches.
Processed 145/634 batches.
Processed 150/634 batches.
Processed 155/634 batches.
Processed 160/634 batches.
Processed 165/634 batches.
Processed 170/634 batches.
Processed 175/634 batches.
Processed 180/634 batches.
Processed 185/634 batches.
Processed 190/634 bat

INFO:utils.ml:Feature extraction completed. Extracted 15195 features.


Processed 634/634 batches.


In [18]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

rf1 = RandomForestClassifier(n_estimators=100)
rf2 = RandomForestClassifier(n_estimators=150)
rf3 = RandomForestClassifier(n_estimators=250)

xgb1 = XGBClassifier(n_estimators=250)
xgb2 = XGBClassifier(n_estimators=350)
xgb3 = XGBClassifier(n_estimators=450)

pipeline.classifiers = [rf1, rf2, rf3,
                        # svm1, svm2, svm3, svm6, svm7, svm8,
                        xgb1, xgb2, xgb3]

In [19]:
pipeline.fit_classifiers()

INFO:utils.ml:Fitting classifiers...
INFO:utils.ml:Fitting classifier: RandomForestClassifier0
INFO:utils.ml:Fitted classifier: RandomForestClassifier0; Done in 22.863141536712646 seconds
INFO:utils.ml:Fitting classifier: RandomForestClassifier1
INFO:utils.ml:Fitted classifier: RandomForestClassifier1; Done in 35.46800994873047 seconds
INFO:utils.ml:Fitting classifier: RandomForestClassifier2
INFO:utils.ml:Fitted classifier: RandomForestClassifier2; Done in 59.546653747558594 seconds
INFO:utils.ml:Fitting classifier: XGBClassifier3
INFO:utils.ml:Fitted classifier: XGBClassifier3; Done in 9.264241695404053 seconds
INFO:utils.ml:Fitting classifier: XGBClassifier4
INFO:utils.ml:Fitted classifier: XGBClassifier4; Done in 13.182026863098145 seconds
INFO:utils.ml:Fitting classifier: XGBClassifier5
INFO:utils.ml:Fitted classifier: XGBClassifier5; Done in 16.433799028396606 seconds
INFO:utils.ml:Fitting completed in 156.76 seconds.


In [20]:
pipeline.predict_with_classifiers(VAL_PATH)

INFO:utils.ml:Predicting with classifiers on dataset: C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\val


Processed 5/159 batches.
Processed 10/159 batches.
Processed 15/159 batches.
Processed 20/159 batches.
Processed 25/159 batches.
Processed 30/159 batches.
Processed 35/159 batches.
Processed 40/159 batches.
Processed 45/159 batches.
Processed 50/159 batches.
Processed 55/159 batches.
Processed 60/159 batches.
Processed 65/159 batches.
Processed 70/159 batches.
Processed 75/159 batches.
Processed 80/159 batches.
Processed 85/159 batches.
Processed 90/159 batches.
Processed 95/159 batches.
Processed 100/159 batches.
Processed 105/159 batches.
Processed 110/159 batches.
Processed 115/159 batches.
Processed 120/159 batches.
Processed 125/159 batches.
Processed 130/159 batches.
Processed 135/159 batches.
Processed 140/159 batches.
Processed 145/159 batches.
Processed 150/159 batches.
Processed 155/159 batches.


INFO:utils.ml:Predictions made with classifier: RandomForestClassifier0
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier1


Processed 159/159 batches.


INFO:utils.ml:Predictions made with classifier: RandomForestClassifier2
INFO:utils.ml:Predictions made with classifier: XGBClassifier3
INFO:utils.ml:Predictions made with classifier: XGBClassifier4
INFO:utils.ml:Predictions made with classifier: XGBClassifier5


{'GT': array([0, 0, 0, ..., 1, 1, 1]),
 'RandomForestClassifier0': array([0, 0, 1, ..., 0, 0, 1]),
 'RandomForestClassifier1': array([0, 0, 1, ..., 0, 0, 1]),
 'RandomForestClassifier2': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier3': array([0, 0, 0, ..., 0, 0, 1]),
 'XGBClassifier4': array([0, 0, 0, ..., 1, 0, 1]),
 'XGBClassifier5': array([0, 0, 0, ..., 0, 0, 1])}

In [22]:
import pandas as pd

df = pd.DataFrame(data=pipeline.calculate_metrics())
df.T

INFO:utils.ml:Metrics for classifier GT: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}
INFO:utils.ml:Metrics for classifier RandomForestClassifier0: {'accuracy': 0.8011064278187566, 'precision': 0.8012165555899041, 'recall': 0.8011064278187566, 'f1': 0.8011209637398138}
INFO:utils.ml:Metrics for classifier RandomForestClassifier1: {'accuracy': 0.8016332982086407, 'precision': 0.8018315239845138, 'recall': 0.8016332982086407, 'f1': 0.8016477948270808}
INFO:utils.ml:Metrics for classifier RandomForestClassifier2: {'accuracy': 0.804004214963119, 'precision': 0.8041782193283306, 'recall': 0.804004214963119, 'f1': 0.8040189057549869}
INFO:utils.ml:Metrics for classifier XGBClassifier3: {'accuracy': 0.7992623814541623, 'precision': 0.7992543354824547, 'recall': 0.7992623814541623, 'f1': 0.799256361178513}
INFO:utils.ml:Metrics for classifier XGBClassifier4: {'accuracy': 0.8016332982086407, 'precision': 0.8016246771683114, 'recall': 0.8016332982086407, 'f1': 0.8016262610408437

,accuracy,precision,recall,f1
GT,1.000000,1.000000,1.000000,1.000000
RandomForestClassifier0,0.801106,0.801217,0.801106,0.801121
RandomForestClassifier1,0.801633,0.801832,0.801633,0.801648
RandomForestClassifier2,0.804004,0.804178,0.804004,0.804019
XGBClassifier3,0.799262,0.799254,0.799262,0.799256
XGBClassifier4,0.801633,0.801625,0.801633,0.801626
XGBClassifier5,0.804795,0.804786,0.804795,0.804781


In [23]:
pipeline.get_feature_names()

['mean_rgb_channel_0',
 'mean_rgb_channel_1',
 'mean_rgb_channel_2',
 'std_rgb_channel_0',
 'std_rgb_channel_1',
 'std_rgb_channel_2',
 'var_rgb_channel_0',
 'var_rgb_channel_1',
 'var_rgb_channel_2',
 'mean_lab_channel_0',
 'mean_lab_channel_1',
 'mean_lab_channel_2',
 'std_lab_channel_0',
 'std_lab_channel_1',
 'std_lab_channel_2',
 'var_lab_channel_0',
 'var_lab_channel_1',
 'var_lab_channel_2',
 'mean_hsv_channel_0',
 'mean_hsv_channel_1',
 'mean_hsv_channel_2',
 'std_hsv_channel_0',
 'std_hsv_channel_1',
 'std_hsv_channel_2',
 'var_hsv_channel_0',
 'var_hsv_channel_1',
 'var_hsv_channel_2',
 'lbp_rad1_bins8_0',
 'lbp_rad1_bins8_1',
 'lbp_rad1_bins8_2',
 'lbp_rad1_bins8_3',
 'lbp_rad1_bins8_4',
 'lbp_rad1_bins8_5',
 'lbp_rad1_bins8_6',
 'lbp_rad1_bins8_7',
 'lbp_rad1_bins8_8',
 'lbp_rad1_bins8_9',
 'lbp_rad1_bins16_0',
 'lbp_rad1_bins16_1',
 'lbp_rad1_bins16_2',
 'lbp_rad1_bins16_3',
 'lbp_rad1_bins16_4',
 'lbp_rad1_bins16_5',
 'lbp_rad1_bins16_6',
 'lbp_rad1_bins16_7',
 'lbp_rad1_

In [30]:
pipeline.save_feature_matrix_to_excel()

INFO:utils.ml:Saving feature matrix to Excel...
INFO:utils.ml:Feature matrix saved to ./features_smoothing_clahe_pad2square_resize_hair_removal_01_norm.xlsx


1730311099.264234

In [29]:
pipeline.get_preprocessing_steps()

{'smoothing': {'kernel_size': 3},
 'clahe': {'clip_limit': 3, 'tile_grid_size': (8, 8)},
 'pad2square': {'fill': nan},
 'resize': {'size': (150, 150)},
 'hair_removal': {},
 '01_norm': {}}